In [1]:
!pip install optuna > /dev/null 2>&1

# 🧠 Optimización de Hiperparámetros en Modelos de Machine Learning

## 🎯 Objetivo de la clase

En esta práctica aprenderemos a **ajustar los hiperparámetros** de un modelo para mejorar su rendimiento.
Exploraremos tres enfoques diferentes:

1. **Grid Search** — búsqueda exhaustiva de combinaciones.
2. **Random Search** — búsqueda aleatoria eficiente.
3. **Bayesian Optimization (Optuna)** — búsqueda inteligente guiada por resultados previos.
---

## 🧩 ¿Qué son los hiperparámetros?

Los **parámetros** son los valores que el modelo aprende automáticamente durante el entrenamiento (por ejemplo, los pesos de una regresión lineal).
Los **hiperparámetros**, en cambio, los definimos nosotros antes del entrenamiento, y controlan el comportamiento del modelo.

Ejemplos de hiperparámetros comunes:
- Número de árboles (`n_estimators`) en un Random Forest
- Profundidad máxima (`max_depth`)
- Tasa de aprendizaje (`learning_rate`)
- Regularización (`C`, `alpha`, `lambda`)

Optimizar hiperparámetros busca **maximizar una métrica de rendimiento** (accuracy, F1, RMSE, etc.) mediante pruebas sistemáticas o guiadas.

---

In [4]:
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.datasets import load_iris
import time
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split, StratifiedKFold, ParameterGrid, ParameterSampler, cross_val_score
from sklearn import tree
from sklearn.metrics import roc_auc_score
from tqdm.notebook import tqdm

import optuna

# importando la base de datos
from sklearn.datasets import make_classification
X, y = make_classification(
    n_samples=2000, n_features=20,
    n_informative=8, n_redundant=4,
    n_clusters_per_class=2,
    class_sep=1.2, flip_y=0.05,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## 🔍 1. Grid Search

**Grid Search** explora **todas las combinaciones posibles** de hiperparámetros dentro de un rango definido.

Es el método más simple y garantiza encontrar el mejor modelo **si la grilla es suficientemente densa**, pero tiene un costo computacional alto.

🧩 **Ventajas**
- Fácil de entender e implementar.
- Evalúa todas las combinaciones posibles.

⚠️ **Desventajas**
- Escala mal con el número de hiperparámetros.
- Puede ser muy costoso en tiempo.

📈 **Ejemplo:** probamos diferentes combinaciones de `n_estimators`, `max_depth` y `min_samples_split`.

### Ejecucion de Grid Search

In [6]:
def auc_safe(y_true, proba):
    """
    Si es binario: usa proba de clase positiva.
    Si es multiclase: usa OVR macro.
    """
    if proba.ndim == 1 or proba.shape[1] == 1:
        return np.nan
    if proba.shape[1] == 2:
        return roc_auc_score(y_true, proba[:, 1])
    else:
        return roc_auc_score(y_true, proba, multi_class="ovr", average="macro")

In [ ]:
# definimos hiperparametros
hyperparameters = {
    "criterion": ["gini", "entropy"],
    "max_depth": [None, 3, 5, 8, 12],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 5],
    "max_features": [None, "sqrt"],
}

# semilla de aleatoriedad
RANDOM_STATE = 42
splits = 5
cv = StratifiedKFold(n_splits=splits, shuffle=True, random_state=RANDOM_STATE)

# GRID SEARCH
learners = []
records = []
grid = ParameterGrid(hyperparameters)

for params in tqdm(grid, total = len(list(grid))):
    fold_scores = []
    fold_idx = 0
    for tr_idx, va_idx in cv.split(X_train, y_train):
        X_tr, X_va = X_train[tr_idx], X_train[va_idx]
        y_tr, y_va = y_train[tr_idx], y_train[va_idx]

        clf = tree.DecisionTreeClassifier(**params, random_state=RANDOM_STATE)
        clf.fit(X_tr, y_tr)

        va_proba = clf.predict_proba(X_va)
        fold_auc = auc_safe(y_va, va_proba)
        fold_scores.append(float(fold_auc))
        fold_idx += 1

    # tomo la media y desvio estandar de los fold de validacion
    cv_mean = float(np.mean(fold_scores))
    cv_std  = float(np.std(fold_scores))

    # Modelo de arbol de decision y clasificacion
    learner = tree.DecisionTreeClassifier(**params, random_state=RANDOM_STATE).fit(X_train, y_train)
    test_proba = learner.predict_proba(X_test)
    test_auc = float(auc_safe(y_test, test_proba))

    learners.append(learner)

    params_json = json.dumps(params, sort_keys=True)
    records.append({
        "params_json": params_json,           # hiperparámetros como string
        "cv_mean_auc": cv_mean,               # promedio AUC CV (5 folds)
        "cv_std_auc": cv_std,                 # desvío AUC CV
        "test_auc": test_auc,                 # AUC en test (hold-out 33%)
        "n_leaves": getattr(learner.tree_, "node_count", None),
        "max_depth_model": learner.get_depth(),
        "fold_scores": json.dumps(fold_scores)  # lista de AUC por fold como string (opcional)
    })

  0%|          | 0/180 [00:00<?, ?it/s]

In [ ]:
results_df = pd.DataFrame(records)

results_df = results_df.sort_values(["cv_mean_auc", "test_auc"], ascending=False).reset_index(drop=True)

print("Top 10 por CV_mean_auc:")
display(results_df.head(10))

best_by_cv = results_df.iloc[0]
print("\nMejor por CV_mean_auc:")
print(best_by_cv)

best_by_test = results_df.sort_values("test_auc", ascending=False).iloc[0]
print("\nMejor por TEST_auc:")
print(best_by_test)

Top 10 por CV_mean_auc:


,params_json,cv_mean_auc,cv_std_auc,test_auc,n_leaves,max_depth_model,fold_scores
0,"{""criterion"": ""gini"", ""max_depth"": 5, ""max_fea...",0.828536,0.024804,0.819278,55,5,"[0.8040671565625639, 0.8382322923045519, 0.800..."
1,"{""criterion"": ""gini"", ""max_depth"": 5, ""max_fea...",0.828536,0.024804,0.819278,55,5,"[0.8040671565625639, 0.8382322923045519, 0.800..."
2,"{""criterion"": ""gini"", ""max_depth"": 5, ""max_fea...",0.828536,0.024804,0.819278,55,5,"[0.8040671565625639, 0.8382322923045519, 0.800..."
3,"{""criterion"": ""gini"", ""max_depth"": 5, ""max_fea...",0.826429,0.023801,0.817366,55,5,"[0.8048836497244335, 0.8317003470095938, 0.808..."
4,"{""criterion"": ""gini"", ""max_depth"": 8, ""max_fea...",0.825975,0.019460,0.823980,153,8,"[0.795596040008165, 0.8315472545417433, 0.8137..."
5,"{""criterion"": ""gini"", ""max_depth"": 8, ""max_fea...",0.825975,0.019460,0.823980,153,8,"[0.795596040008165, 0.8315472545417433, 0.8137..."
6,"{""criterion"": ""gini"", ""max_depth"": 8, ""max_fea...",0.825975,0.019460,0.823980,153,8,"[0.795596040008165, 0.8315472545417433, 0.8137..."
7,"{""criterion"": ""gini"", ""max_depth"": 5, ""max_fea...",0.825847,0.025274,0.817822,59,5,"[0.8048836497244335, 0.8330016329863238, 0.802..."
8,"{""criterion"": ""gini"", ""max_depth"": 5, ""max_fea...",0.825771,0.025136,0.817822,59,5,"[0.8048836497244335, 0.8330016329863238, 0.802..."
9,"{""criterion"": ""entropy"", ""max_depth"": 5, ""max_...",0.824730,0.024162,0.804366,55,5,"[0.7956981016533986, 0.8593845682792407, 0.804..."



Mejor por CV_mean_auc:
params_json        {"criterion": "gini", "max_depth": 5, "max_fea...
cv_mean_auc                                                 0.828536
cv_std_auc                                                  0.024804
test_auc                                                    0.819278
n_leaves                                                          55
max_depth_model                                                    5
fold_scores        [0.8040671565625639, 0.8382322923045519, 0.800...
Name: 0, dtype: object

Mejor por TEST_auc:
params_json        {"criterion": "entropy", "max_depth": 5, "max_...
cv_mean_auc                                                 0.780486
cv_std_auc                                                  0.053065
test_auc                                                     0.84127
n_leaves                                                          61
max_depth_model                                                    5
fold_scores        [0.7140998162890

### Otra forma de ejecutar un grid search

In [ ]:
clf = tree.DecisionTreeClassifier(**params, random_state=RANDOM_STATE)

grid_search = GridSearchCV(clf, hyperparameters, cv=3, scoring='roc_auc', n_jobs=-1)
start = time.time()
grid_search.fit(X_train, y_train)
end = time.time()

print(f"⏱️ Tiempo total: {end - start:.2f} s")
print(f"Mejores parámetros: {grid_search.best_params_}")
print(f"Mejor accuracy CV: {grid_search.best_score_:.3f}")

⏱️ Tiempo total: 15.28 s
Mejores parámetros: {'criterion': 'entropy', 'max_depth': 8, 'max_features': None, 'min_samples_leaf': 5, 'min_samples_split': 2}
Mejor accuracy CV: 0.822


### Ejecucion de Random Search

In [7]:
# hiperparametros de random forest
# random search no toma todos los valores, sino muestras aleatorias de esta lista
hyperparameters = {
    "criterion": ["gini", "entropy"],
    "max_depth": [None, 3, 5, 8, 12],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 5],
    "max_features": [None, "sqrt"],
}

# Semilla random
RANDOM_STATE = 42

# 5 fold cross validation
# Estratificada = mantiene la proporción de clases en cada fold.
splits = 5
cv = StratifiedKFold(n_splits=splits, shuffle=True, random_state=RANDOM_STATE)

learners = []   # guarda cada modelo entrenado
records = []    # guarda los resultados

# ParameterSampler toma el diccionario de hiperparámetros y genera N combinaciones aleatorias.
# Aquí se generan 30 combinaciones al azar.
random = ParameterSampler(hyperparameters, n_iter = 30)
# random search en la maxima es igual a grid search, en la minima es mucho mas rapido

# Recorremos cada combinación aleatoria de hiperparámetros
for params in tqdm(random, total = len(list(random))):
    fold_scores = []    # AUC de cada fold
    fold_idx = 0

    # Validacion cruzada
    for tr_idx, va_idx in cv.split(X_train, y_train):
        # Crear sets de train y validacion segun los indices
        X_tr, X_va = X_train[tr_idx], X_train[va_idx]
        y_tr, y_va = y_train[tr_idx], y_train[va_idx]

        # instanciar modelo decision tree
        clf = tree.DecisionTreeClassifier(**params, random_state=RANDOM_STATE)
        clf.fit(X_tr, y_tr)
        # probabilidades del fold de validacion
        va_proba = clf.predict_proba(X_va)
        # ROC-AUC del fold
        fold_auc = auc_safe(y_va, va_proba)
        fold_scores.append(float(fold_auc))
        fold_idx += 1

    # Promedio de metricas de los 5 folds
    cv_mean = float(np.mean(fold_scores))
    cv_std  = float(np.std(fold_scores))

    # Se entrena un modelo con TODO el training set con el set de parametros de esta vuelta
    learner = tree.DecisionTreeClassifier(**params, random_state=RANDOM_STATE).fit(X_train, y_train)
    # Evualuar en el set de test
    test_proba = learner.predict_proba(X_test)
    test_auc = float(auc_safe(y_test, test_proba))
    # guardo el modelo
    learners.append(learner)
    # guardamos resultados y metricas
    params_json = json.dumps(params, sort_keys=True)
    records.append({
        "params_json": params_json,           # hiperparámetros como string
        "cv_mean_auc": cv_mean,               # promedio AUC CV (5 folds)
        "cv_std_auc": cv_std,                 # desvío AUC CV
        "test_auc": test_auc,                 # AUC en test (hold-out 33%)
        "n_leaves": getattr(learner.tree_, "node_count", None),
        "max_depth_model": learner.get_depth(),
        "fold_scores": json.dumps(fold_scores)  # lista de AUC por fold como string (opcional)
    })

  0%|          | 0/30 [00:00<?, ?it/s]

In [8]:
# Mostrando los resultados
results_df = pd.DataFrame(records)

results_df = results_df.sort_values(["cv_mean_auc", "test_auc"], ascending=False).reset_index(drop=True)

print("Top 10 por CV_mean_auc:")
display(results_df.head(10))

best_by_cv = results_df.iloc[0]
print("\nMejor por CV_mean_auc:")
print(best_by_cv)

best_by_test = results_df.sort_values("test_auc", ascending=False).iloc[0]
print("\nMejor por TEST_auc:")
print(best_by_test)

Top 10 por CV_mean_auc:


,params_json,cv_mean_auc,cv_std_auc,test_auc,n_leaves,max_depth_model,fold_scores
0,"{""criterion"": ""entropy"", ""max_depth"": 5, ""max_...",0.824730,0.024162,0.804366,55,5,"[0.7956981016533986, 0.8593845682792407, 0.804..."
1,"{""criterion"": ""entropy"", ""max_depth"": 5, ""max_...",0.822596,0.028986,0.811102,53,5,"[0.7787303531332925, 0.8656358440498062, 0.807..."
2,"{""criterion"": ""entropy"", ""max_depth"": 5, ""max_...",0.822596,0.028986,0.811102,53,5,"[0.7787303531332925, 0.8656358440498062, 0.807..."
3,"{""criterion"": ""gini"", ""max_depth"": 12, ""max_fe...",0.811824,0.023907,0.798369,213,12,"[0.8036333945703205, 0.808532353541539, 0.7737..."
4,"{""criterion"": ""gini"", ""max_depth"": 12, ""max_fe...",0.811824,0.023907,0.798369,213,12,"[0.8036333945703205, 0.808532353541539, 0.7737..."
5,"{""criterion"": ""gini"", ""max_depth"": 3, ""max_fea...",0.803649,0.042513,0.756086,15,3,"[0.7513268013880383, 0.815753214941825, 0.7612..."
6,"{""criterion"": ""gini"", ""max_depth"": 3, ""max_fea...",0.803582,0.042580,0.756086,15,3,"[0.7513268013880383, 0.815753214941825, 0.7608..."
7,"{""criterion"": ""entropy"", ""max_depth"": 12, ""max...",0.803108,0.032051,0.812736,221,12,"[0.7833741579914268, 0.834736680955297, 0.7951..."
8,"{""criterion"": ""gini"", ""max_depth"": 3, ""max_fea...",0.801954,0.044678,0.756086,15,3,"[0.7428556848336395, 0.815753214941825, 0.7612..."
9,"{""criterion"": ""gini"", ""max_depth"": 8, ""max_fea...",0.797004,0.019649,0.825780,169,8,"[0.8118238416003265, 0.7858236374770362, 0.769..."



Mejor por CV_mean_auc:
params_json        {"criterion": "entropy", "max_depth": 5, "max_...
cv_mean_auc                                                  0.82473
cv_std_auc                                                  0.024162
test_auc                                                    0.804366
n_leaves                                                          55
max_depth_model                                                    5
fold_scores        [0.7956981016533986, 0.8593845682792407, 0.804...
Name: 0, dtype: object

Mejor por TEST_auc:
params_json        {"criterion": "entropy", "max_depth": 5, "max_...
cv_mean_auc                                                 0.789855
cv_std_auc                                                   0.04367
test_auc                                                     0.84127
n_leaves                                                          61
max_depth_model                                                    5
fold_scores        [0.7185905286793

## 🔮 El Surrogate Model en la Optimización Bayesiana

La **optimización bayesiana** se basa en la idea de **aprender una representación aproximada** de la función que queremos optimizar.  
Esa representación se llama **surrogate model** (modelo sustituto o aproximador).

---

### 🎯 ¿Qué problema resuelve?

Cada vez que entrenamos un modelo para probar una combinación de hiperparámetros, el costo puede ser **muy alto** (minutos u horas).  
La idea del surrogate es **evitar probar todo**, aprendiendo una **versión barata** de la función objetivo.

\[
f(\text{hiperparámetros}) = \text{métrica del modelo (por ejemplo, F1 o AUC)}
\]

El surrogate intenta aproximar esta función \( f(x) \) a partir de los puntos que ya conocemos.

---

### 🧠 ¿Qué hace el surrogate model?

1. **Aprende** una relación entre los hiperparámetros (`x`) y el resultado (`f(x)`).
2. **Predice** qué combinaciones podrían dar buenos resultados.
3. **Informa a la función de adquisición** (acquisition function) dónde conviene probar después.

💬 En otras palabras:
> El surrogate model es como un “mapa” aproximado del rendimiento del modelo en función de los hiperparámetros.

---

### 🧩 ¿Cómo funciona el proceso completo?

1. **Exploración inicial:**  
   Se prueban algunas combinaciones aleatorias para tener puntos de referencia.

2. **Ajuste del surrogate model:**  
   Con esos puntos (hiperparámetros → score), el surrogate aprende a estimar:
   - El valor esperado del score (media).  
   - La incertidumbre de esa predicción (varianza).

3. **Selección de nuevos puntos:**  
   Una *acquisition function* (como *Expected Improvement*) elige nuevos hiperparámetros balanceando:
   - **Exploración:** probar zonas donde el surrogate no tiene datos.  
   - **Explotación:** concentrarse en las zonas donde predice alto rendimiento.

4. **Actualización:**  
   Se entrena el modelo real en ese nuevo punto → se obtiene un nuevo score → se actualiza el surrogate.

---

### ⚙️ Tipos comunes de surrogate models

| Modelo | Descripción | Cuándo se usa |
|---------|--------------|----------------|
| **Gaussian Process (GP)** | Modela la función objetivo como una distribución gaussiana sobre funciones posibles. Proporciona media y varianza. | Problemas pequeños y continuos. |
| **Tree-structured Parzen Estimator (TPE)** | Modela distribuciones de probabilidad sobre los parámetros buenos y malos. | Es el que usa **Optuna** y **Hyperopt**, eficiente en espacios grandes. |
| **Random Forest surrogate** | Usa un bosque aleatorio para aproximar el comportamiento de la función. | Casos con mucho ruido o hiperparámetros mixtos. |

---

### 🧮 Cómo trabaja TPE (usado por Optuna)

Optuna implementa el método **Tree-structured Parzen Estimator (TPE)**, que:

1. Divide las observaciones en dos grupos:
   - **Buenos resultados** (por encima de un umbral de score).
   - **Malos resultados** (por debajo de ese umbral).

2. Aprende dos distribuciones:
   - \( l(x) \): probabilidad de que una combinación provenga del grupo “bueno”.
   - \( g(x) \): probabilidad de que provenga del grupo “malo”.

3. Elige nuevas combinaciones maximizando la razón \( l(x) / g(x) \),  
   es decir, **donde es más probable encontrar mejoras**.

💬 En criollo:
> “Aprende cómo se ven los hiperparámetros que funcionaron bien,  
> y busca combinaciones parecidas, pero sin dejar de explorar nuevas zonas.”

---

### 🗺️ Intuición visual

Imaginá que querés escalar una montaña 🏔️, pero solo podés medir la altura en unos pocos puntos:

- **Grid Search** → mide la altura en todos los puntos del mapa (caro).  
- **Random Search** → mide en puntos al azar (barato, pero sin dirección).  
- **Bayesiana (con surrogate)** → usa las mediciones anteriores para construir un **mapa de alturas estimadas**  
  y luego decide dónde conviene medir la próxima vez (más cerca de la cima esperada).

---

### 🔄 Ciclo completo de la optimización bayesiana

| Paso | Qué hace | Quién lo hace |
|------|-----------|---------------|
| 1️⃣ Probar combinaciones iniciales | Entrena y evalúa modelos reales | `objective()` |
| 2️⃣ Aprender la relación entre hiperparámetros y score | Ajusta el surrogate model | Optuna |
| 3️⃣ Elegir nuevas combinaciones prometedoras | Usa una función de adquisición | Optuna |
| 4️⃣ Reentrenar y actualizar | Registra el nuevo punto (x, f(x)) | Optuna |

---

### 💬 Cómo explicarlo en clase

Podés usar esta analogía:
> “El surrogate model es como un mapa que se va completando a medida que tomamos mediciones.  
> Al principio el mapa es difuso, pero con cada nueva medición el modelo entiende mejor  
> dónde están las zonas altas (buenas combinaciones).”

---

### 🧭 En resumen

| Concepto | Rol |
|-----------|-----|
| **Surrogate model** | Aprende una versión aproximada de la función objetivo (score). |
| **Objective function** | Evalúa el rendimiento real del modelo (costoso). |
| **Acquisition function** | Decide dónde probar después (balance exploración/explotación). |
| **Optuna (TPE)** | Implementa todo el ciclo de forma automática y eficiente. |

---

### 💡 Frase para recordar
> “El surrogate model no entrena el modelo final,  
> entrena una **intuición estadística** de qué combinaciones de hiperparámetros pueden rendir mejor.”

---

### 🎯 ¿Qué hace la función `objective`?

Cada vez que Optuna realiza una prueba (*trial*), ejecuta `objective(trial)` con una combinación de hiperparámetros.  
La función:

1. **Recibe** un conjunto de hiperparámetros propuestos por Optuna.  
2. **Entrena** un modelo con esos valores.  
3. **Evalúa** el modelo (por ejemplo, con cross-validation).  
4. **Devuelve** una métrica numérica que mide el rendimiento (accuracy, F1, AUC, etc.).

Ese valor numérico se conoce como **objective value**, y es lo que Optuna trata de **maximizar o minimizar**.

---

### 🧠 Ejemplo práctico

```python
def objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    max_depth = trial.suggest_int("max_depth", 3, 15)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 5)
    max_features = trial.suggest_categorical("max_features", [None, "sqrt", "log2"])

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42
    )

    score = cross_val_score(model, X, y, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
    return score

In [ ]:
#SCORING = "f1_macro"
SCORING = "roc_auc"

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

#primero definimos una funcion objetivo
def objective(trial: optuna.Trial) -> float:
    """
    Optuna probará distintas combinaciones de hiperparámetros y
    evaluará el promedio de F1 macro en CV.
    """
    # 🔧 Hiperparámetros a explorar
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    max_depth = trial.suggest_int("max_depth", 3, 15)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 5)
    max_features = trial.suggest_categorical("max_features", [None, "sqrt", "log2"])

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=RANDOM_STATE
    )

    # ⚙️ Cross-validation con la misma CV
    score = cross_val_score(model, X_train, y_train, cv=cv, scoring=SCORING, n_jobs=-1).mean()

    return score  # Optuna maximiza este valor





# 🧪 Validación cruzada fija (para comparación justa)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)



In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30, show_progress_bar=True)

print("🏆 Mejor F1 CV promedio:", study.best_value)
print("✨ Mejores hiperparámetros encontrados:")
print(study.best_params)

[I 2025-11-08 00:00:06,354] A new study created in memory with name: no-name-14382b6b-3901-49db-b7b8-cb84500a3036


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2025-11-08 00:00:14,368] Trial 0 finished with value: 0.8975709328434375 and parameters: {'n_estimators': 248, 'max_depth': 5, 'min_samples_split': 9, 'min_samples_leaf': 4, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.8975709328434375.
[I 2025-11-08 00:00:24,933] Trial 1 finished with value: 0.9133394570320472 and parameters: {'n_estimators': 121, 'max_depth': 14, 'min_samples_split': 8, 'min_samples_leaf': 3, 'max_features': None}. Best is trial 1 with value: 0.9133394570320472.
[I 2025-11-08 00:00:28,408] Trial 2 finished with value: 0.8684731577873036 and parameters: {'n_estimators': 211, 'max_depth': 3, 'min_samples_split': 3, 'min_samples_leaf': 4, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.9133394570320472.
[I 2025-11-08 00:00:32,996] Trial 3 finished with value: 0.88601755460298 and parameters: {'n_estimators': 295, 'max_depth': 4, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.9133394570320472.
[I

In [ ]:
optuna.visualization.plot_optimization_history(study)


In [ ]:
optuna.visualization.plot_param_importances(study)